In [1]:
import numpy as np
from pickle import load, dump
import os

# Change names


In [2]:
a0 = [0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.975]
an1 = [0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.975]

alpha = [(a,b) for a in a0 for b in an1 if a <= b]

for folder in ["Results", "Performance metrics"]:
    for (a,b) in alpha:
        for strat in ["stat","dyn"]:

            old_file = os.path.join(f"../Experiments/{folder}/", f"Age_service_level_{a,b}_stat-{strat}")
            new_file = os.path.join(f"../Experiments/{folder}/", f"Age_service_level_{a,b}_0_1_stat-{strat}")
            
            os.rename(old_file, new_file)


# Demand break for FIFO stat-stat

In [7]:
file = open("C:/Users/ari_r/OneDrive - Universidad de los andes/Montreal 2024-10/stoch_LSP_PP/Experiments/Parameters/Demand_T20_S150_n4", "rb")
d = load(file); file.close()

file = open("C:/Users/ari_r/OneDrive - Universidad de los andes/Montreal 2024-10/stoch_LSP_PP/Experiments/Parameters/Global_T20_S150_n4", "rb")
(T, S, G, n, h, c, f, C) = load(file); file.close()

In [10]:
compl_scenarios = [d[ix][s] for ix in d for s in S]
compl_scenarios = compl_scenarios[:600]

In [16]:
scenarios = 20; reps = 30

new_demand = {ix:{(s+1):compl_scenarios[ix*scenarios+s] for s in range(scenarios)} for ix in range(reps)}

In [20]:
file = open("C:/Users/ari_r/OneDrive - Universidad de los andes/Montreal 2024-10/stoch_LSP_PP/Experiments/Parameters/Demand_T20_S150_n4_FIFO", "wb")
dump(new_demand,file); file.close()
